In [ ]:
# --- Mount Google Drive ---
from google.colab import drive
drive.mount('/content/drive')

# --- Install necessary packages ---
!pip install ultralytics gdown pandas -q

# --- Import libraries ---
import os
import time
import numpy as np
import tensorflow as tf
from ultralytics import YOLO
import gdown
import pandas as pd
from tensorflow.keras.applications import VGG19, MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam

print("✅ Setup complete.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Setup complete.


In [ ]:
# --- Configuration ---
# Main dataset directory in your Google Drive
DATA_DIR = '/content/drive/MyDrive/bisindo_dataset_split'
train_dir = os.path.join(DATA_DIR, 'train')
val_dir = os.path.join(DATA_DIR, 'val')
test_dir = os.path.join(DATA_DIR, 'test')

# Model and Image Parameters
IMG_SIZE = 224
BATCH_SIZE = 32
NUM_CLASSES = 26 # Based on your training notebooks
NUM_TEST_IMAGES = 160
EPOCHS = 15 # Set a reasonable number of epochs for training

# To store final results
comparison_results = []

In [ ]:
# --- Load Datasets for TensorFlow ---
print("Loading TensorFlow datasets...")
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

# Early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# --- 1. TRAIN AND BENCHMARK MOBILENETV2 (Correct Architecture) ---
print("\n======================================")
print("     Training MobileNetV2")
print("======================================")

# Build Model
base_mobilenet = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')
base_mobilenet.trainable = False

# CORRECTED: This now matches your original MobileNetV2 notebook
mobilenet_model = Sequential([
    tf.keras.layers.Rescaling(1./255),
    base_mobilenet,
    GlobalAveragePooling2D(),
    Dense(512, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.5), # Correct Dropout for MobileNetV2
    Dense(NUM_CLASSES, activation='softmax')
])
mobilenet_model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train Model
mobilenet_model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=[early_stopping])
mobilenet_model.save('mobilenetv2_best.h5')
print("✅ MobileNetV2 model trained and saved.")

# Benchmark Model
print("\n--- Benchmarking MobileNetV2 ---")
test_ds_inf = tf.keras.utils.image_dataset_from_directory(
    test_dir, seed=123, image_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, shuffle=False
)
inference_samples = test_ds_inf.take(NUM_TEST_IMAGES // BATCH_SIZE)
# GPU Warm-up
for images, _ in inference_samples.take(1): _ = mobilenet_model.predict(images, verbose=0)
# Timed Inference
total_time, num_images = 0, 0
for images, _ in inference_samples:
    start_time = time.time()
    _ = mobilenet_model.predict(images, verbose=0)
    end_time = time.time()
    total_time += (end_time - start_time)
    num_images += len(images)
avg_time_ms = (total_time / num_images) * 1000
comparison_results.append({
    'Model': 'MobileNetV2',
    'Parameters': f"{mobilenet_model.count_params():,}",
    'Inference Time (ms/image)': f"{avg_time_ms:.4f}"
})
print(f"✅ MobileNetV2 Benchmark Done: {avg_time_ms:.4f} ms/image")

Loading TensorFlow datasets...
Found 3126 files belonging to 26 classes.
Using 2501 files for training.
Found 670 files belonging to 26 classes.
Using 134 files for validation.

     Training MobileNetV2
Epoch 1/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 942s 12s/step - accuracy: 0.0559 - loss: 4.7119 - val_accuracy: 0.1269 - val_loss: 3.7085
Epoch 2/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 8s 98ms/step - accuracy: 0.1764 - loss: 3.7299 - val_accuracy: 0.2836 - val_loss: 3.2636
Epoch 3/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 9s 108ms/step - accuracy: 0.3113 - loss: 3.1999 - val_accuracy: 0.3507 - val_loss: 2.9693
Epoch 4/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 8s 99ms/step - accuracy: 0.4007 - loss: 2.7862 - val_accuracy: 0.4104 - val_loss: 2.7627
Epoch 5/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 8s 100ms/step - accuracy: 0.4444 - loss: 2.6186 - val_accuracy: 0.4776 - val_loss: 2.6197
Epoch 6/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 8s 106ms/step - accuracy: 0.5063 - loss: 2.3864 - val_accuracy: 0.4851 - val_loss: 2.5293
Epoch 7/15
79/79 ━━━━━━━━━━━━━

✅ MobileNetV2 model trained and saved.

--- Benchmarking MobileNetV2 ---
Found 670 files belonging to 26 classes.
✅ MobileNetV2 Benchmark Done: 3.0759 ms/image


In [ ]:
# --- 2. TRAIN AND BENCHMARK VGG19 (Correct Architecture) ---
print("\n======================================")
print("         Training VGG19")
print("======================================")

# Build Model
base_vgg19 = VGG19(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')
base_vgg19.trainable = False

# CORRECTED: This now matches your original VGG19 notebook
vgg19_model = Sequential([
    tf.keras.layers.Rescaling(1./255),
    base_vgg19,
    GlobalAveragePooling2D(),
    Dense(512, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.4), # Correct Dropout for VGG19
    Dense(NUM_CLASSES, activation='softmax')
])
vgg19_model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train Model
vgg19_model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=[early_stopping])
vgg19_model.save('vgg19_best.h5')
print("✅ VGG19 model trained and saved.")

# Benchmark Model
print("\n--- Benchmarking VGG19 ---")
# GPU Warm-up
for images, _ in inference_samples.take(1): _ = vgg19_model.predict(images, verbose=0)
# Timed Inference
total_time, num_images = 0, 0
for images, _ in inference_samples:
    start_time = time.time()
    _ = vgg19_model.predict(images, verbose=0)
    end_time = time.time()
    total_time += (end_time - start_time)
    num_images += len(images)
avg_time_ms = (total_time / num_images) * 1000
comparison_results.append({
    'Model': 'VGG19',
    'Parameters': f"{vgg19_model.count_params():,}",
    'Inference Time (ms/image)': f"{avg_time_ms:.4f}"
})
print(f"✅ VGG19 Benchmark Done: {avg_time_ms:.4f} ms/image")


         Training VGG19
80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 48s 359ms/step - accuracy: 0.0555 - loss: 4.2118 - val_accuracy: 0.0224 - val_loss: 3.7974
Epoch 2/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 18s 223ms/step - accuracy: 0.1257 - loss: 3.6610 - val_accuracy: 0.0448 - val_loss: 3.6939
Epoch 3/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 18s 226ms/step - accuracy: 0.2112 - loss: 3.3419 - val_accuracy: 0.1269 - val_loss: 3.5618
Epoch 4/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 18s 233ms/step - accuracy: 0.2630 - loss: 3.0763 - val_accuracy: 0.2090 - val_loss: 3.3788
Epoch 5/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 18s 231ms/step - accuracy: 0.3292 - loss: 2.8842 - val_accuracy: 0.3284 - val_loss: 3.1613
Epoch 6/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 18s 228ms/step - accuracy: 0.3663 - loss: 2.6971 - val_accuracy: 0.3731 - val_loss: 2.9430
Epoch 7/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 18s 227ms/step - accuracy: 0.4010 - loss: 2.5652 - val_accuracy: 0.3657 - val_loss: 2.7691
Epoch 8/15
79/79 ━━━━

✅ VGG19 model trained and saved.

--- Benchmarking VGG19 ---
✅ VGG19 Benchmark Done: 8.5736 ms/image


In [ ]:
# --- 3. DOWNLOAD AND BENCHMARK YOLOv8 ---
print("\n======================================")
print("        Benchmarking YOLOv8")
print("=======================================")

# --- Your YOLOv8 Model File Path from Google Drive ---
# NOTE: Using a direct path is better since your Drive is already mounted.
# Make sure this path is correct for your Google Drive setup.
yolov8_model_path = '/content/drive/My Drive/Colab Notebooks/best.pt' # <--- VERIFY THIS PATH

# Check if the file exists
if not os.path.exists(yolov8_model_path):
    raise FileNotFoundError(f"YOLOv8 model not found at: {yolov8_model_path}. Please verify the path.")

# Load Model
print("Loading YOLOv8 model...")
yolo_model = YOLO(yolov8_model_path)
print("✅ YOLOv8 model loaded.")


# !! CORRECTON: Explicitly move the model to the GPU !!
print("Moving model to GPU...")
yolo_model.to('cuda')
print("✅ Model moved to GPU.")

# Standard params for yolov8n-cls
yolo_params = 1468186

# Prepare Image Paths
test_image_paths = []
for root, _, files in os.walk(test_dir):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            test_image_paths.append(os.path.join(root, file))
sample_image_paths = test_image_paths[:NUM_TEST_IMAGES]

# GPU Warm-up
if sample_image_paths:
    _ = yolo_model.predict(sample_image_paths[0], verbose=False)

# Timed Inference on GPU
print("Running timed inference on GPU...")
start_time = time.time()
_ = yolo_model.predict(sample_image_paths, verbose=False)
end_time = time.time()
total_time = end_time - start_time
avg_time_ms = (total_time / len(sample_image_paths)) * 1000

comparison_results.append({
    'Model': 'YOLOv8n-cls',
    'Parameters': f"{yolo_params:,}",
    'Inference Time (ms/image)': f"{avg_time_ms:.4f}"
})
print(f"✅ YOLOv8 Benchmark Done: {avg_time_ms:.4f} ms/image")


        Benchmarking YOLOv8
Loading YOLOv8 model...
✅ YOLOv8 model loaded.
Moving model to GPU...
✅ Model moved to GPU.
Running timed inference on GPU...
✅ YOLOv8 Benchmark Done: 5.2247 ms/image


In [ ]:
# --- Display Results Table ---
print("\n======================================")
print("     Final Model Comparison")
print("======================================")

if comparison_results:
    df = pd.DataFrame(comparison_results)
    # Ensure consistent column order
    df = df[['Model', 'Parameters', 'Inference Time (ms/image)']]
    print(df.to_string(index=False))
else:
    print("No results were generated.")

print("======================================\n")


     Final Model Comparison
      Model Parameters Inference Time (ms/image)
MobileNetV2  2,929,242                    3.0759
      VGG19 20,302,426                    8.5736
YOLOv8n-cls  1,468,186                  774.0325
YOLOv8n-cls  1,468,186                    5.2247

